In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import toolbox
import plotly as py
import plotly.figure_factory as ff
print(dir(toolbox.notebook))
from collections import Counter
from utils import get_hero_from_id, proper_match, all_valid_players
from toolbox.notebook import get_json_lines_in_folder, progress_bar

py.offline.init_notebook_mode(connected=True)

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'fs', 'get_files_in_folders', 'get_json_lines_in_folder', 'json', 'np', 'os', 'plt', 'progress_bar', 'ready_image', 'show_image']


In [ ]:
import json

matches = []

for match in get_json_lines_in_folder('./scraper/scrape_results/'):
    if match.get('type', 'match') == 'match' and proper_match(match) and all_valid_players(match):
        matches.append(match)

HBox(children=(IntProgress(value=0, max=22), HTML(value='')))

In [30]:
match['players'][0].keys()

dict_keys(['account_id', 'hero_healing', 'backpack_2', 'scaled_tower_damage', 'scaled_hero_damage', 'item_1_name', 'leaver_status', 'hero_name', 'item_0_name', 'gold_per_min', 'item_3_name', 'kills', 'hero_id', 'item_2_name', 'assists', 'item_2', 'item_4', 'deaths', 'xp_per_min', 'leaver_status_description', 'scaled_hero_healing', 'item_1', 'tower_damage', 'backpack_0', 'gold_spent', 'last_hits', 'item_4_name', 'item_5', 'backpack_1', 'item_3', 'gold', 'player_slot', 'hero_damage', 'item_0', 'item_5_name', 'level', 'ability_upgrades', 'leaver_status_name', 'denies'])

In [3]:
from collections import defaultdict

friendly_matchups = defaultdict(lambda : defaultdict(lambda: (0,0)))
hostile_matchups = defaultdict(lambda : defaultdict(lambda: (0,0)))

def divide_players_by_team(players):
    return ([x['hero_id'] for x in players if x['player_slot'] < 127],
            [x['hero_id'] for x in players if x['player_slot'] > 127])

for match in get_json_lines_in_folder('./scraper/scrape_results/'):
    if match.get('type', 'match') != 'match' or proper_match(match) is False or all_valid_players(match) is False:
        continue
    radient_win = match['radiant_win']
    radient_players, dire_players = divide_players_by_team(match['players'])
    # Win flag is for knowing if it was a win for that team
    for team, opposing_team, win_flag in [(radient_players, dire_players, True), 
                                          (dire_players, radient_players, False)]:
        for friendly in team:
            if friendly == 0:
                raise ValueError
            # Opposing_matchup
            opponent_matchups = hostile_matchups[friendly]
            for opponent in opposing_team:
                existing_wins, existing_count = opponent_matchups[opponent]
                if radient_win == win_flag:
                    existing_wins += 1
                opponent_matchups[opponent] = (existing_wins, existing_count + 1)
            hostile_matchups[friendly] = opponent_matchups
    
            #Friendly teamups
            teammate_matchups = friendly_matchups[friendly]
            for team_mate in team:
                if friendly != team_mate or True:
                    existing_wins, existing_count = teammate_matchups.get(opponent, (0, 0))
                    if radient_win == win_flag:
                        existing_wins += 1
                    teammate_matchups[opponent] = (existing_wins, existing_count + 1)
            friendly_matchups[friendly] = teammate_matchups

HBox(children=(IntProgress(value=0, max=22), HTML(value='')))

In [4]:
heroes = sorted(friendly_matchups.keys())
from utils import get_hero_from_id

def get_score(matchups, current_hero, other_hero):
    wins, total = matchups[current_hero][other_hero]
    if current_hero == other_hero:
        return 0
    try:
        return wins/total
    except ZeroDivisionError:
        return 0.5 # We've never seen this matchup. Mostly happens if operating on smaller dataset
    
def get_label(matchups, current_hero, other_hero):
    score = get_score(matchups, current_hero, other_hero)
    return f"{get_hero_from_id(current_hero)['name']} - {get_hero_from_id(other_hero)['name']} - {score}"

heroes = heroes[0:20]

friendly_matchup_graph = [[get_score(friendly_matchups, current_hero, matchup_hero) for matchup_hero in heroes] for current_hero in heroes]
opposing_matchup_graph = [[get_score(hostile_matchups, current_hero, matchup_hero) for matchup_hero in heroes] for current_hero in heroes]

friendly_labels = [[get_label(friendly_matchups, current_hero, matchup_hero) for matchup_hero in heroes] for current_hero in heroes]
opposing_labels = [[get_label(hostile_matchups, current_hero, matchup_hero) for matchup_hero in heroes] for current_hero in heroes]

annotations = [['' for matchup_hero in heroes] for current_hero in heroes]

sidebars = [get_hero_from_id(current_hero)['name'] for current_hero in heroes]

# Make Annotated Heatmap
pt = ff.create_annotated_heatmap(friendly_matchup_graph, x=sidebars, y=sidebars, annotation_text=annotations, text=friendly_labels, hoverinfo='text')
pt.layout.title = 'Friendly Matchups - Y is Hero, X is win rate'

py.offline.iplot(pt, filename='periodic_table')
# import matplotlib.pyplot as plt

# plt.imshow(friendly_matchup_graph, cmap='hot', interpolation='nearest')
# plt.show()

# plt.imshow(opposing_matchup_graph, cmap='hot', interpolation='nearest')
# plt.show()

In [12]:
# Just for fun, best and worst friendly matchups

def generate_perc(values):
    if values[1] == 0:
        return 0.5
    return values[0]/values[1]

def convert_to_percs(matchups):
    return {k: {k2: generate_perc(v2) for k2, v2 in v.items()} for k,v in matchups.items()}

friendly_matchups_percs = convert_to_percs(friendly_matchups)

print('Friendly Matchups')
for h1, matchups in friendly_matchups_percs.items():
    best_score, best_matchup = max([(score, x) for x, score in matchups.items()])
    worst_score, worst_matchup = min([(score, x) for x, score in matchups.items()])
    print(f"{get_hero_from_id(h1)['name']} - Best: {get_hero_from_id(best_matchup)['name']} - {best_score}")
    print(f"{get_hero_from_id(h1)['name']} - Worst: {get_hero_from_id(worst_matchup)['name']} - {worst_score}")

Friendly Matchups
rubick - Best: lone_druid - 0.6451612903225806
rubick - Worst: riki - 0.35062611806797855
phantom_assassin - Best: lone_druid - 0.7117117117117117
phantom_assassin - Worst: spectre - 0.3882752761257434
phoenix - Best: sand_king - 0.6742081447963801
phoenix - Worst: huskar - 0.4031413612565445
antimage - Best: lone_druid - 0.6428571428571429
antimage - Worst: chen - 0.4107142857142857
tusk - Best: lone_druid - 0.7435897435897436
tusk - Worst: centaur - 0.32460732984293195
storm_spirit - Best: lone_druid - 0.625
storm_spirit - Worst: undying - 0.4
slark - Best: gyrocopter - 0.6111111111111112
slark - Worst: zuus - 0.3842756183745583
undying - Best: templar_assassin - 0.7131147540983607
undying - Worst: chen - 0.4090909090909091
tidehunter - Best: pangolier - 0.6140350877192983
tidehunter - Worst: chen - 0.3333333333333333
silencer - Best: lone_druid - 0.6440677966101694
silencer - Worst: chen - 0.3902439024390244
windrunner - Best: lone_druid - 0.7099236641221374
windru

NameError: name 'best_friendly_matchup' is not defined

In [13]:
hostile_matchups_percs = convert_to_percs(hostile_matchups)

print('Hostile Matchups')
for h1, matchups in hostile_matchups_percs.items():
    best_score, best_matchup = max([(score, x) for x, score in matchups.items()])
    worst_score, worst_matchup = min([(score, x) for x, score in matchups.items()])
    print(f"{get_hero_from_id(h1)['name']} - Best: {get_hero_from_id(best_matchup)['name']} - {best_score}")
    print(f"{get_hero_from_id(h1)['name']} - Worst: {get_hero_from_id(worst_matchup)['name']} - {worst_score}")


Hostile Matchups
rubick - Best: wisp - 0.5783132530120482
rubick - Worst: spectre - 0.36515291936978683
phantom_assassin - Best: lone_druid - 0.6501831501831502
phantom_assassin - Worst: spectre - 0.3831000518403318
phoenix - Best: lone_druid - 0.6632124352331606
phoenix - Worst: abaddon - 0.43521594684385384
antimage - Best: puck - 0.6117775354416576
antimage - Worst: riki - 0.43182944967773923
tusk - Best: lone_druid - 0.6185567010309279
tusk - Worst: spectre - 0.3464611872146119
storm_spirit - Best: furion - 0.5965865011636928
storm_spirit - Worst: spectre - 0.3951048951048951
slark - Best: lone_druid - 0.6591422121896162
slark - Worst: zuus - 0.38988300068823123
undying - Best: batrider - 0.6829268292682927
undying - Worst: riki - 0.46059865607819184
tidehunter - Best: gyrocopter - 0.6006240249609984
tidehunter - Worst: elder_titan - 0.4065934065934066
silencer - Best: lone_druid - 0.6349809885931559
silencer - Worst: spectre - 0.42597722376115726
windrunner - Best: lone_druid - 0.

In [16]:
import json

def perc_and_normalize(matchups):
    return {int(k): {int(k2): generate_perc(v2)-0.5 for k2, v2 in v.items()} for k,v in matchups.items()}

# Save the matchups to a pair of files
with open('./friendly_matchups.json', 'wb') as fileref:
    fileref.write(json.dumps(perc_and_normalize(friendly_matchups), indent=4).encode('utf8'))
with open('./enemy_matchups.json', 'wb') as fileref:
    fileref.write(json.dumps(perc_and_normalize(hostile_matchups), indent=4).encode('utf8'))